In [9]:
import psycopg2
import pandas as pd 
# from sqlalchemy.types import Integer, Text, String, DateTime
import sqlalchemy as s
import matplotlib
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import json

with open("config.json") as config_file:
    config = json.load(config_file)

database_connection_string = 'postgres+psycopg2://{}:{}@{}:{}/{}'.format(config['user'], config['password'], config['host'], config['port'], config['database'])

dbschema='augur_data'
engine = s.create_engine(
    database_connection_string,
    connect_args={'options': '-csearch_path={}'.format(dbschema)})

In [10]:
repo_list = pd.DataFrame()
repo_list_query = f"""
SELECT repo_id, repo_name, repo_path from repo
WHERE repo_name = 'concourse' OR repo_name = 'postfacto' or repo_name = 'clarity' or repo_name = 'gpdb' or
      repo_name = 'kpack' or repo_name = 'rabbitmq-server' or repo_name = 'sonobuoy' or repo_name = 'velero' 
      or repo_name = 'octant' or repo_name = 'gpdb' or repo_name = 'tern' or repo_name = 'rabbitmq-perf-test'
      or repo_name = 'charts' or repo_name = 'pks-kubo-release' or repo_name = 'cf-rabbitmq-release';
    """
repo_list = pd.read_sql_query(repo_list_query, con=engine)
print(repo_list)

    repo_id            repo_name                              repo_path
0     28051            concourse                  github.com/concourse/
1     27940                 tern                 github.com/tern-tools/
2     26235            concourse               github.com/pcfdev-forks/
3     27809   rabbitmq-perf-test    github.com/pivotal-cf-experimental/
4     27913              clarity                     github.com/vmware/
5     30039               charts                    github.com/bitnami/
6     25551   rabbitmq-perf-test                   github.com/rabbitmq/
7     26232  cf-rabbitmq-release               github.com/pcfdev-forks/
8     26983            postfacto                    github.com/pivotal/
9     27043                kpack                    github.com/pivotal/
10    27235  cf-rabbitmq-release                 github.com/pivotal-cf/
11    28118               charts                  github.com/concourse/
12    27169                 gpdb                github.com/pivot

In [11]:
## List of repository IDs for the report
#repo_dict = {25760, 25663} #spring-boot & spring-framework
#repo_dict = {28051} # concourse
#repo_dict = {26983} #postfacto
#repo_dict = {25432} #rabbitmq-server
#repo_dict = {25663} #spring-framework
#repo_dict = {28030} #sonobuoy
#repo_dict = {28031} #velero
#repo_dict = {27913} #clarity
#repo_dict = {28037} #octant
#repo_dict = {25857} #gpdb
repo_dict = {27235}

In [57]:
#from datetime import date
import datetime 

current = datetime.date.today()
today = "'" + str(current) + "'"
#print(current)

first_current = current.replace(day=1)
last_month = first_current - datetime.timedelta(days=1)
end_date = "'" + str(last_month) + "'"
print(end_date)

start = last_month - datetime.timedelta(days=365)
start_date = "'" + str(start) + "'"
print(start_date)



'2020-10-31'
'2019-11-01'


In [59]:
from datetime import datetime, timezone

end_dt = datetime.strptime(end_date, "'%Y-%m-%d'").replace(tzinfo=timezone.utc)
print(end_dt)

start_dt = datetime.strptime(start_date, "'%Y-%m-%d'").replace(tzinfo=timezone.utc)
print(start_dt)



2020-10-31 00:00:00+00:00
2019-11-01 00:00:00+00:00


In [41]:
from datetime import datetime, timedelta, timezone

#now = datetime.now(timezone.utc)
#current = now.replace(hour=0, minute=0, second=0, microsecond=0)

#end_dt = current.replace(day=1) - timedelta(days=1)
print(end_dt)

#start_dt = end_dt - timedelta(days=365)
print(start_dt)

2020-10-31 00:00:00+00:00
2019-11-01 00:00:00+00:00


In [60]:
#Commit data - from humans excluding known bots
commitsDF = pd.DataFrame()
for value in repo_dict: 
    commitsquery = f"""
                    SELECT
                        DISTINCT(cmt_commit_hash),
                        contributors.cntrb_canonical,
                        canonical_full_names.cntrb_full_name AS canonical_full_name,
                        cmt_author_name, cmt_author_email, repo_id, cmt_author_timestamp 
                    FROM commits 
                        LEFT OUTER JOIN contributors ON cntrb_email = cmt_author_email left outer join 
                        (
                            SELECT distinct on (cntrb_canonical) cntrb_full_name, cntrb_canonical, data_collection_date
                            FROM contributors
                            WHERE cntrb_canonical = cntrb_email
                            order by cntrb_canonical
                        ) canonical_full_names on canonical_full_names.cntrb_canonical = contributors.cntrb_canonical
                    WHERE
                        repo_id = {value}
                        AND cmt_author_name NOT LIKE 'snyk%%'
                        AND cmt_author_name NOT LIKE '%%bot'
                        AND cmt_author_name NOT LIKE '%%Bot'
                        AND cmt_author_name != 'Bitnami Containers'
                        AND cmt_author_name != 'Spring Operator'
                        AND cmt_author_name != 'Spring Buildmaster'
--                         AND cmt_author_timestamp >= {start_date}
--                         AND cmt_author_timestamp <= {end_date}
                    ORDER BY
                        cntrb_canonical;
                    """
    
    get_repo_name_query = f"""
        SELECT repo_name from repo where repo_id = {value};
           """
    get_repo_name = pd.read_sql_query(get_repo_name_query, con=engine)
    repo_name = get_repo_name.repo_name[0]
    
all_commitsDF = pd.read_sql_query(commitsquery, con=engine)
commitsDF = all_commitsDF[(all_commitsDF['cmt_author_timestamp'] > start_dt) & (all_commitsDF['cmt_author_timestamp'] < end_dt)]
total_commits = commitsDF.cmt_commit_hash.nunique()  

In [61]:
authorDF = pd.DataFrame()
authorDF = commitsDF.cntrb_canonical.value_counts()
authorDF = authorDF.reset_index()
authorDF.columns = ['email', 'commits']
authorDF['percent'] = authorDF['commits'] / total_commits
print(authorDF.head(10))
print()

                        email  commits   percent
0    cf-london-eng@pivotal.io      150  0.635593
1         david.ansari@gmx.de       36  0.152542
2        alexblease@gmail.com       11  0.046610
3        mirah.gary@gmail.com       10  0.042373
4  harsha.chaudhary@gmail.com       10  0.042373
5           fjilla@pivotal.io        7  0.029661
6        gtatarzyn@pivotal.io        5  0.021186
7        mkuratczyk@gmail.com        4  0.016949
8      support@dependabot.com        2  0.008475
9        chunyineko@gmail.com        1  0.004237



In [62]:
authorDF = pd.DataFrame()
#authorDF = commitsDF.canonical_full_name.value_counts()
authorDF = commitsDF.cmt_author_name.value_counts()
authorDF = authorDF.reset_index()
authorDF.columns = ['name', 'commits']
authorDF['percent'] = authorDF['commits'] / total_commits
print(authorDF.head(10))

                   name  commits   percent
0  Cloud Foundry London       94  0.398305
1           CI Pipeline       56  0.237288
2          David Ansari       36  0.152542
3           Alex Blease       22  0.093220
4      Harsha Chaudhary       20  0.084746
5            Mirah Gary       20  0.084746
6           Feroz Jilla        7  0.029661
7     Grzegorz Tatarzyn        5  0.021186
8      Michal Kuratczyk        4  0.016949
9            Chunyi Lyu        2  0.008475


In [63]:
import textwrap

cum_percent = 0
people_list = []

i = 1
num_people = 0

for item in authorDF.iterrows():
    name = item[1]['name']
    percent = item[1]['percent']
    commits = item[1]['commits']
    
    cum_percent += percent
    
    people_list.append([name, percent, commits])
    
    if (cum_percent > .70 and num_people == 0):
        num_people = i
        risk_percent = cum_percent
        
    if i == 8:
        if cum_percent <= .70:
            risk_percent = cum_percent
            num_people = i
        break
    i+=1
    
risk_list = []
bar_colors = []

j = 1
for person in people_list:
    name = person[0]
    if len(name) > 15:
        new_name = textwrap.wrap(name, 15)
        name = '\n'.join(new_name)
    percent = person[1]
    commits = person[2]
    risk_list.append([name, percent, commits])
    
    if (num_people < 3 and j <= num_people):
        bar_colors.append('red')
    else:
        bar_colors.append('lightblue')
    
    j+=1
    

names = [item[0] for item in risk_list]
percents = [item[1] for item in risk_list]
commits = [item[2] for item in risk_list]

sns.set_style('ticks')
sns.set(style="whitegrid", font_scale=2)

fig, ax = plt.subplots()

# the size of A4 paper
fig.set_size_inches(24, 8)

title = repo_name + "\nContributor Risk Metric Assessment: "

if num_people < 3:
    title += "AT RISK"
    title_color = 'firebrick'
else:
    title += "Healthy"
    title_color = 'forestgreen'
title += "\n" + str(num_people) + " people made up " + "{:.0%}".format(risk_percent) + " of the commits in the past year.\n"

risk_bar = sns.barplot(x=names, y=commits, palette=bar_colors).set_title(title, fontsize=30, color=title_color)

risk_bar_labels = ax.set_xticklabels(names, wrap=True)
risk_bar_labels = ax.set_ylabel('Commits')
risk_bar_labels = ax.set_xlabel('\nKey Contributors\n\nA healthy project should have at a minimum 3 people who combined account for the majority (>50%) of the commits.\nThe higher this number is, the more likely your project would succeed if a leading contributor suddenly left the project.')

i = 0
for p in ax.patches:
    ax.annotate("{:.0%}".format(percents[i]), (p.get_x() + p.get_width() / 2., p.get_height()),
        ha='center', va='center', color='gray', xytext=(0, 20),
        textcoords='offset points')
    i+=1


In [31]:
from pathlib import Path
Path("output/tmp/tmp2").mkdir(parents=True, exist_ok=True)
fig.savefig('output/tmp/tmp2/test1.png')

## Misc data exploration
Trying to understand what's happening with the multiple canonical names

In [42]:
test = commitsDF.loc[commitsDF['cntrb_canonical'] == 'jrosland@vmware.com'].canonical_full_name
print(test[402])

Jonas Rosland


In [48]:
commitsDF.loc[commitsDF['cntrb_canonical'] == 'bryanliles@gmail.com']

cmt_commit_hash       cntrb_canonical  \
28   005fba92f6b43beab5b788c4803c154da87160f1  bryanliles@gmail.com   
29   00b5c25e697375d3a58c0eabcfb53e878ab6f060  bryanliles@gmail.com   
30   00fac7d8aa415772fdbd69af800a4b36e98865a2  bryanliles@gmail.com   
31   02c9ffaa4350b832aca7dd050911ea24d2bd9cbf  bryanliles@gmail.com   
32   02e4c929af1e21cfb45774804ff5e94a018a6aa4  bryanliles@gmail.com   
..                                        ...                   ...   
377  fc3a26d05f70cb2be80246d5f8ea26562fb03976  bryanliles@gmail.com   
378  fcb5a2ca48210c6c108cb165421fcdfdc88c5b29  bryanliles@gmail.com   
379  fd44145165a6e47f86d3e51bc9536130cd019258  bryanliles@gmail.com   
380  fe36a3665e587de53f20cf551e86729f9066b972  bryanliles@gmail.com   
381  ff717236c2f6ced6b2c5e540fe51f2198b839dd0  bryanliles@gmail.com   

    canonical_full_name cmt_author_name      cmt_author_email  repo_id  \
28               bryanl          bryanl  bryanliles@gmail.com    28037   
29          Bryan Liles     Bryan Liles  bryanliles@gmail.com    28037   
30               bryanl          bryanl  bryanliles@gmail.com    28037   
31               lilesb          lilesb  bryanliles@gmail.com    28037   
32               bryanl          bryanl  bryanliles@gmail.com    28037   
..                  ...             ...                   ...      ...   
377              bryanl          bryanl  bryanliles@gmail.com    28037   
378         Bryan Liles     Bryan Liles  bryanliles@gmail.com    28037   
379         Bryan Liles     Bryan Liles  bryanliles@gmail.com    28037   
380         Bryan Liles     Bryan Liles  bryanliles@gmail.com    28037   
381              bryanl          bryanl  bryanliles@gmail.com    28037   

         cmt_author_timestamp  
28  2019-04-04 00:09:16+00:00  
29  2019-04-18 19:07:00+00:00  
30  2019-08-03 13:51:51+00:00  
31  2019-09-15 20:23:59+00:00  
32  2019-06-11 18:51:13+00:00  
..                        ...  
377 2019-03-13 19:26:45+00:00  
378 2020-02-20 14:07:57+00:00  
379 2019-04-08 16:09:13+00:00  
380 2019-07-22 17:11:18+00:00  
381 2019-06-10 20:01:19+00:00  

[354 rows x 7 columns]